# Senior Challengers: Lane Lines Detection using OpenCV

In [118]:
#Importing all necessary libraries
import cv2 
import numpy as np
import imutils
import matplotlib.pyplot as plt
import time 
import sys

In [119]:
# Command line argument 1st = Input video file name 2nd = Output video file name 3rd = Output contour file name
args = sys.argv

if len(args) == 1:
    sys.exit(1)
    
input_file = args[1]

output_file = None
contour_file = None
if len(args) >= 3:
    output_file = args[2]
if len(args) == 4:
    contour_file = args[3]
prev_left_avg = [0.000001, 0] 
prev_right_avg = [0.000001, 0] 
direction = "Straight" 

In [120]:
def avg_slope_intercept(img, lines):
    left_points = []
    right_points = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)

            parameters = np.polyfit((x1, x2), (y1, y2), 1)
            slope = parameters[0]
            intercept = parameters[1]

            # we can differentiate lines whether on right or lefft by their slope values:
            if slope < 0:  # points on the left have negative slope
                left_points.append((slope, intercept))  # so we add these points to our left_points array
            else:
                right_points.append((slope, intercept))  # else these points are on the right side

            # to draw the continuous line we have to find averages of these left or right points arrays
            left_points_avg = np.average(left_points,
                                         axis=0)  # axis should be 0 because we want averages of columns in array
            right_points_avg = np.average(right_points, axis=0)

            # we need the coordinates to draw the line:
            right_line = make_coordinates(img, right_points_avg)
            left_line = make_coordinates(img, left_points_avg)

            return np.array([left_line, right_line])

In [121]:
def extract_yellow_white(frame):
    # Apply masks for white and yellow lines
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # the limits for white color
    lower_white = np.array([0, 0, 100], dtype=np.uint8) # the lower limit
    upper_white = np.array([255, 255, 255], dtype=np.uint8) # the upper limit
    # the mask of the only white color of the frame
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    
    # the limits of yellow color
    lower_yellow = np.array([20, 100, 100], dtype=np.uint8) # the lower limit
    upper_yellow = np.array([30, 255, 255], dtype=np.uint8) # the upper limit
    # the mask of the only yellow color of the frame
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # merge both masks into one mask
    mask = cv2.bitwise_or(mask_white, mask_yellow)
    # apply the masks to the frame
    frame = cv2.bitwise_and(frame, frame, mask=mask)
    
    # return the frame with only white and yellow colors
    return frame

In [122]:
def canny_image(frame):
    # frame -> gray image 
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    # GaussianBlur
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    # Convert the frame to BINARY image (black and white)
    binary = cv2.threshold(blur, 110, 255, cv2.THRESH_BINARY)[1]
    canny = cv2.Canny(binary, 100, 150)
    
    return canny

In [123]:
# function to extract only the lower part of the image
def roi(frame):
    
    height = frame.shape[0] # height
    width = frame.shape[1] # width
    # polygon 
    polygons = np.array([
                            [(5 * width//16, height), 
                             (6 * width // 16, 7 * height//9),  
                             (10 * width // 16, 7 * height//9), 
                             (14*width//16, height)] 
                        ])
    
    mask = np.zeros_like(frame)
    # polygons filling
    cv2.fillPoly(mask, polygons, 255)
    segment = cv2.bitwise_and(frame, mask)

    return segment

In [125]:
# function to identify the lines on the frames from Hough Lines 
def measure_l(frame, lines):
    global prev_left_avg, prev_right_avg 
    if lines is None:
        return frame
    
    left = []
    right = []

    for line in lines:
        # Reshapes line from 2D array to 1D array of size 4
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        # extract the slope and y-intecept separately
        slope = parameters[0]
        y_intercept = parameters[1]
   
        if slope < 0:
            left.append((slope, y_intercept)) # left line
        else:
            right.append((slope, y_intercept)) # right line
        
    # average of the left and right lines into a single slope and y-intercept value
    left_avg = np.average(left, axis = 0) # the slope
    right_avg = np.average(right, axis = 0) # the y-incercept

    if type(left_avg) == np.ndarray:
        prev_left_avg = left_avg
    else: 
        left_avg = prev_left_avg

    if type(right_avg) == np.ndarray:
        prev_right_avg = right_avg
    else: 
        right_avg = prev_right_avg
        
    # Find the x1, y1, x2, y2 coordinates for the left and right lines
    left_line = coordinates(frame, left_avg)
    right_line = coordinates(frame, right_avg)

    return np.array([left_line, right_line])

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [126]:
#coordinates of the lines in the frame
def coordinates(frame, line_params):
    # extract the slope and y-intercept from the line
    try:
        slope, intercept = line_params
    except:
        slope, intercept = 0, 0
   
    if slope == 0:
        y1 = frame.shape[0]
        y2 = frame.shape[0]
        x1 = frame.shape[1] / 2
        x2 = frame.shape[1] / 2
    else: 
        y1 = frame.shape[0]
       
        y2 = int(y1 - frame.shape[0]*0.2)
        x1 = int((y1 - intercept) / slope)
        x2 = int((y2 - intercept) / slope)
    
    return np.array([x1, y1, x2, y2])

In [127]:
# draw the lines on the rame
def visualize_lines(frame, lines):
    # Create an empty image of the same size as the frame filled with the zeros
    lines_visualize = np.zeros_like(frame)
    f_points = [] # find the points for drawing the colored polygon
    
    # Create an empty image of the same size as the frame filled with the zeros
    mask = np.zeros_like(frame)
    
    # if there are no lines, return the empty frame without any changes 
    if lines is None:
        return frame
    
    # loop through each set of coordinates as the lines 
    for x1, y1, x2, y2 in lines:
        # Draws lines between two coordinates with green color and 5 thickness
        cv2.line(lines_visualize, (x1, y1), (x2, y2), (0, 200, 0), 5)
        # append the points of the lines for the rectangle
        f_points.append((x2, y2)) 
        f_points.append((x1, y1))

    # create an empty array to rearrange the points of the polygon 
    points = []
    points.append(f_points[0]) 
    points.append(f_points[1])
    points.append(f_points[3])
    points.append(f_points[2])

    # draw the polygon with the given points with filled with red color on the mask
    cv2.fillPoly(mask, np.array([points]), (0, 0, 255))
    
    # return the frame filled the drawn lines and the polygon on it
    return cv2.bitwise_or(lines_visualize, mask)

In [128]:

# function to find the direction of the car movement 
def find_direction(lines):
    global direction 
    slopes = []

    if lines is None:
        return direction
    if len(lines) > 2:
        return direction
 
    for x1, y1, x2, y2 in lines.reshape(2, 4):
        slopes.append(np.arctan((y2-y1) / (x2-x1)) * 180 / np.pi)

    if slopes[0] < -55 and slopes[1] < 41:
        direction = "left" # left direction
    elif slopes[0] > -50 and slopes[1] > 48:
        direction = "right" # right direction
    else:
        direction = "straight" # default direction


    return direction

In [130]:
def video(w, h):
    global direction # use the global variable
    cap = cv2.VideoCapture("input_video.mp4") # read the video named "Road.mp4" 
    fps = cap.get(cv2.CAP_PROP_FPS) # find the FPS of the video
    
    # while the camera is openned 
    while (cap.isOpened()):
        # store the starting time before the frame processing
        start_time = time.time()
        ret, frame = cap.read()

        if ret:
            try: 
                frame = cv2.resize(frame, (w, h))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                frame[:,:,2] -= 20
                frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
                extract = extract_yellow_white(frame)
                
                canny = canny_image(extract)

                # region of interest
                segment = roi(canny)
                
                # hough lines 
                hough = cv2.HoughLinesP(segment, 1, np.pi / 180, 25, np.array([]), minLineLength = 25, maxLineGap = 50)           
                lines = measure_l(frame, hough)

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                # make it lighter by rising the brightness by 20
                frame[:,:,2] += 20
                # convert back to BGR color spaces
                frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
       
                direction = find_direction(lines)
                # Drawing the lines on the frame
                lines_visualize = visualize_lines(frame, lines)

                output = cv2.addWeighted(frame, 1, lines_visualize, 1, 1)
            except:
                # the case if some error occurs during the execution of the functions
                output = frame
        else:
            # the case if the video has ended
            break
    
        # store the ending time after the frame processing
        end_time = time.time()
        text = "DIRECTION: {}".format(direction.upper())
        org = (10, 25)
        fontFace = cv2.FONT_HERSHEY_PLAIN
        fontScale = 1
        color = (0, 0, 255)
        thickness = 1
        output = cv2.putText(output, text, org, fontFace, fontScale, color, thickness)
        # FPS  
        text = "FPS: {}".format(int(1/(end_time - start_time)))
        org = (10, 50)

        # put the FPS text on the frame
        output = cv2.putText(output, text, org, fontFace, fontScale, color, thickness)

        # Opens a new window and displays the output frame
        cv2.imshow("outout_video.mp4", output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
video(w=640, h=480)